In [1]:
from typing_extensions import TypedDict
from IPython.display import Image, display
from PIL import Image as PILImage
import io
from enum import Enum
from typing import Annotated
import os
import json
import arxiv
from typing import List
from pydantic import BaseModel
from langchain_core.messages import ToolMessage
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AnyMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.types import Command, interrupt

from langgraph.prebuilt import ToolNode, tools_condition
from composio_langgraph import Action, ComposioToolSet, App
from dotenv import load_dotenv
load_dotenv()
os.environ["COMPOSIO_API_KEY"] = os.getenv("COMPOSIO_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages
class State(MessagesState):
    next: str

In [5]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages


class State(MessagesState):
    next: str


def make_research_channel_decider(llm: BaseChatModel, members: list[str]) -> str:
    options = members
    system_prompt = (
        "You are a research channel decider. You will need to decide which"
        "medium of research is best to obtain the information requested by the user."
        "You will be given a prompt and your job is to decide which medium"
        f"from the following list of of options {members} is best to use."
    )

    class Router(TypedDict):
        """Worker to route to next. If no workers needed, route to FINISH."""

        next: Literal[*options]

    def channel_decider_node(state: State) -> Command[Literal[*members]]:
        """An LLM-based router."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]

        return Command(goto=goto, update={"next": goto})

    return channel_decider_node

In [8]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4", temperature=0)
search_agent = create_react_agent(llm , tools=[])


def search_node(state: State) -> Command[Literal["__end__"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # We want our workers to always finish and end their execution cycle
        goto="FINISH",
    )

youtube_agent = create_react_agent(llm, tools=[])
def youtube_node(state: State) -> Command[Literal["__end__"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="youtube")
            ]
        },
        # We want our workers to always finish and end their execution cycle
        goto="FINISH",
    )


research_channel_decider_node = make_research_channel_decider(llm, ["search", "youtube"])

In [10]:
research_builder = StateGraph(State)
research_builder.add_node("channel_decider", research_channel_decider_node)
research_builder.add_node("search", search_node)
research_builder.add_node("youtube", youtube_node)

research_builder.add_edge(START, "channel_decider")
research_graph = research_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(research_graph.get_graph().draw_mermaid_png()))

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 502.